# PENSION DATA ETL

### Import sql queries from sql_queries.py

In [52]:
import psycopg2
import pandas as pd
import os
from sql_queries import create_table_queries, drop_table_queries
from sql_queries import dcplandata_table_insert,  pensiontopstockstopbonds_table_insert, pensioninvestmentfees_table_insert, \
                        pensioncreditrating_table_insert, ppddatalatest_table_insert, pfplan_ppdsupplement_table_insert, \
                        retirementsystemdata_table_insert, pensioninvestmentperformancedetailed_table_insert, \
                        statelocaldisability_ppd_table_insert

### Check current working directory

In [53]:
print(os.getcwd())

C:\Users\user\Documents\idif\idif-etl-with-postgresql


### Create IDiF pension database and connect cursor

In [54]:
conn = psycopg2.connect("user=postgres \
                        host=127.0.0.1 \
                        port=5432 \
                        dbname=postgres \
                        password=student")
cur = conn.cursor()
conn.set_session(autocommit=True)

# create idif_pension_db database
cur.execute("""DROP DATABASE IF EXISTS idif_pension_db""")
cur.execute("""CREATE DATABASE idif_pension_db""")

# close intial connection
conn.close()
cur.close()

# connect to idif_pension_db
conn = psycopg2.connect("user=postgres \
                        host=127.0.0.1 \
                        port=5432 \
                        dbname=idif_pension_db \
                        password=student")
cur = conn.cursor()

### Create database tables

In [55]:
for query in create_table_queries:
    cur.execute(query)
    conn.commit()

### Create a list of integers from 0-27

In [56]:
n = []
for i in range(28):
    n.append(i)
    
print(n)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]


### Read dcplan csv file with pandas dataframe

In [57]:
df = pd.read_csv(r"pension_csv_data\\DCPlanData.csv", encoding = "ISO-8859-1", usecols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27])
df.drop_duplicates(inplace=True)
df.head(3)

,dcplanid,state,planname,fy,year_est,ppd_id,planname_db,primary,cashbalance,hybrid,...,eq_avg,ee_contrate_mandatory,ee_contrate_max,ee_contrate_verbatim,er_contrate_matchrate,er_contrate_fixedrate,er_contrate_verbatim,conts_ee,conts_er,coveredpayroll
0,1,AK,Alaska PERS  DCR Plan,2015,2006.0,3,Alaska PERS,1,NaN,1.0,...,NaN,.08,NaN,NaN,NaN,0.05,NaN,70462.0,38949.0,NaN
1,1,AK,Alaska PERS  DCR Plan,2016,2006.0,3,Alaska PERS,1,NaN,1.0,...,NaN,.08,NaN,NaN,NaN,0.05,NaN,77196.0,43350.0,NaN
2,1,AK,Alaska PERS  DCR Plan,2017,2006.0,3,Alaska PERS,1,NaN,1.0,...,NaN,.08,NaN,NaN,NaN,0.05,NaN,83493.0,46982.0,NaN


### check datatype for each column

In [58]:
df.dtypes

dcplanid                   int64
state                     object
planname                  object
fy                         int64
year_est                 float64
ppd_id                    object
planname_db               object
primary                    int64
cashbalance              float64
hybrid                   float64
mandatory_default        float64
plantype                  object
eligible_eegroups         object
actives                  float64
inactives                float64
members_total            float64
assets                   float64
investmentoptions         object
eq_avg                   float64
ee_contrate_mandatory     object
ee_contrate_max          float64
ee_contrate_verbatim      object
er_contrate_matchrate    float64
er_contrate_fixedrate    float64
er_contrate_verbatim      object
conts_ee                 float64
conts_er                 float64
coveredpayroll           float64
dtype: object

### Insert dcplan data from the dataframe into dcplan table in the database
**Use variable 'a' to track the numbers of rows:** This helps dectect errors

In [59]:
a = 0
try:
    for value in df.values:
        dcplanid, state, planname, fy, year_est, ppd_id, planname_db, primary_, cashbalance, hybrid, mandatory_default, plantype, eligible_eegroups, actives, inactives, members_total, assets, investmentoptions, eq_avg, ee_contrate_mandatory, ee_contrate_max, ee_contrate_verbatim, er_contrate_matchrate, er_contrate_fixedrate, er_contrate_verbatim, conts_ee, conts_er, coveredpayroll = value
        cur.execute(dcplandata_table_insert, (dcplanid, state, planname, fy, year_est, str(ppd_id), planname_db, primary_, cashbalance, hybrid, mandatory_default, plantype, eligible_eegroups, actives, inactives, members_total, assets, investmentoptions, eq_avg, ee_contrate_mandatory, ee_contrate_max, ee_contrate_verbatim, er_contrate_matchrate, er_contrate_fixedrate, er_contrate_verbatim, conts_ee, conts_er, coveredpayroll))
        a += 1
        print(a)
        conn.commit()
except Exception as e:
    print(e)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


### Correct errors using the 'UPDATE TABLE' statement and transform columns using the 'ALTER TABLE' statement

In [60]:
cur.execute("UPDATE dcplandata SET ppd_id = '95' WHERE ppd_id = '95; 96';")
conn.commit()

In [61]:
cur.execute("ALTER TABLE dcplandata ALTER COLUMN ppd_id TYPE float USING ppd_id::float;")
conn.commit()

### Get database schema information

In [62]:
cur.execute("select * from information_schema.tables;")
rows = cur.fetchall()
for row in rows:
    print(row)

('idif_pension_db', 'public', 'retirementsystembasics', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None)
('idif_pension_db', 'public', 'pensiontopstockstopbonds', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None)
('idif_pension_db', 'public', 'pensioninvestmentfees', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None)
('idif_pension_db', 'public', 'pensioncreditrating', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None)
('idif_pension_db', 'public', 'ppddatalatest', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None)
('idif_pension_db', 'public', 'pfplan_ppdsupplement', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None)
('idif_pension_db', 'public', 'retirementsystemdata', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None)
('idif_pension_db', 'public', 'pensioninvestmentperformancedetailed', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None)
('idif_pension_db', 'public', 'statelocaldisabili

### Drop all tables in the database

In [50]:
for query in drop_table_queries:
    cur.execute(query)
    conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


### Close connection

In [51]:
conn.close()
cur.close()